In [ ]:
import spacy

import pandas as pd
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_csv('data/large_all_listings_clean.csv')

In [ ]:
import en_core_web_lg

In [ ]:
nlp = spacy.load("en_core_web_lg", disable=['tagger', 'parser', 'ner'])

In [ ]:
nlp.pipeline

[]

In [ ]:
def lemmatize(texts):
    #texts = texts['description'].tolist()
    try:
        m = nlp(texts)
        lemmas = " ".join([token.lemma_ for token in m if not token.is_stop])
        return lemmas
    except Exception as e:
        print(texts)
        print(e)
        return ""

    

In [ ]:
lemma_descriptions = df['description'].apply(lemmatize)

nan
object of type 'float' has no len()
nan
object of type 'float' has no len()
nan
object of type 'float' has no len()
nan
object of type 'float' has no len()
nan
object of type 'float' has no len()
nan
object of type 'float' has no len()
nan
object of type 'float' has no len()


In [ ]:
lemma_descriptions

0         Description \r\n\r\n Conduent world large prov...
1         PVH Corp. global , action - orient company cha...
2         -Act business partner consult Hiring Managers ...
3         combine company work Hollywood big blockbuster...
4         energy newsroom . pace trade floor . buzz rece...
                                ...                        
150664    Volt Workforce Solutions look Healthcare Sales...
150665    Private Client Banker responsible deliver high...
150666    Education :   Bachelors Degree \r\n Position C...
150667    Responsible Loan Ops team service mortgage loa...
150668    Administrative assistant work VP Publicity , p...
Name: description, Length: 150669, dtype: object

In [ ]:
vectorizer = TfidfVectorizer(stop_words=None,
                             min_df=30, 
                             max_df=.5,
                             max_features=1000)

In [ ]:
tfid_vectors = vectorizer.fit_transform(lemma_descriptions)

In [ ]:
model = KMeans(n_clusters=6, max_iter=300, n_init=10)
model.fit(tfid_vectors)

KMeans(n_clusters=6)

In [ ]:
df['cluster'] = model.labels_

In [ ]:
def finance_clusters(df):
    
    finance_companies = ['Goldman Sachs', 'Bank of America']
    cluser_counts = pd.Series(data= 0, index=[0, 1,2,3,4])

    companies = df.groupby(by='company')

    # This can probably be a apply and merge
    for group, company in companies:
        if group in finance_companies:
            counts = company['cluster'].value_counts()
            
            cluser_counts = cluser_counts.add(counts, fill_value = 0)
    
    threshold = .2
    top = cluser_counts.nlargest(2)
    return top.index.tolist()

finance_clusters(df)


[2, 3]

In [ ]:
def label_companies(df, finance_clusters):
    """
        See if postings related to the company indicate it's a finance company
    """
    df.loc[:,'type'] = ""
    companies = df.groupby(by='company')

    # This can probably be a agg and merge
    for group, company in companies:
        #total_records = company.size()
        c = company['cluster'].value_counts(normalize=True)
        # c = c.nlargest(2)
        top_clusters = [c.idxmax()]
 
        # TODO: put weighting function here

        for c in top_clusters:
            if c in finance_clusters:
                df.loc[df['company'] == group, ['type']] = "finance"
    


In [ ]:
def write_filtered_file(df, just_names=False):
    if just_names:
        finance_c = df.loc[df['type'] == 'finance']['company'].unique()
        pd.Series(finance_c).to_csv('data/list_of_companies.csv', index=False)
    else:
        df.loc[df['type'] == 'finance'][['company','title','description']].to_csv('data/large_only_finance.csv')

In [ ]:
fc = finance_clusters(df)
label_companies(df, fc)

In [ ]:
write_filtered_file(df, just_names=False)

In [ ]:
df['description'].iloc[5]

"JOB SUMMARY\r\nThe Teller is responsible for assisting customers with various types of transactions while effectively maintaining a cash drawer and providing an enjoyable, valuable and memorable customer experience, ensuring prompt, efficient and accurate service. In addition, the Teller engages customers in dialogue to uncover needs for products and services. The hours for this position are Monday- Friday 8-530, Saturday 845-1215. The position is part-time.\r\n\r\nREQUIRED KNOWLEDGE AND SKILLS\r\n\xa0\xa0• High school diploma or equivalent required\r\n\xa0\xa0• 0 - 1 Years experience in function\r\n\xa0\xa0• 0 - 1 Years experience in industry\r\n\xa0\xa0• Prior cash handling and / or customer service experience required.\r\n\xa0\xa0• Basic computer skills using Windows-based software required; prior experience with bank specific software preferred.\r\n\xa0\xa0• Proficient verbal and written communication skills; strong interpersonal skills, with the ability to deliver effective commu

In [ ]:
def investigate(df, company):
    return df.loc[df['company'] == company]


investigate(df, 'Goldman Sachs') 

NameError: name 'df' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=67ebbc4a-1b4b-449c-a00e-722787d292e7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>